# Table of Contents
- 1) [Imports](#imports)   
- 2) [Data Understanding](#data_understanding)   
- 3) [Data Visualization](#data_Visualization)   
- 4) [Data Preprocessing](#data_reprocessing) 
   - 4.1) [Data Cleaning](#data_cleaning)  
   - 4.2) [Features Engineering](#features_engineering)
   - 4.3) [Data Scaling](#data_scaling)
   - 4.4) [Feature Selection](#feature_selection)
- 5) [Clustering](#clustering)
   - 5.1) [DBSCAN](#dbscan)
   - 5.2) [KMeans](#kmeans)
   - 5.3) [KMedoids](#kmedoids)
   - 5.4) [Hierarchical Clustering](#hclustering)
- 6) [EXTRA - Semi-Supervised Learning](#semi_learning)



<div class="alert alert-block alert-info">
<a class="anchor" id="imports"> 
<h1>1. Imports</h1>
</a>    
</div>

In [ ]:
# Packages
import os
import numpy as np
from numpy import percentile
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
import re
import statistics
import scipy
from scipy import stats
import collections
from collections import Counter
# import statsmodels.api as smi
import pylab

from minisom import MiniSom

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.cluster import (DBSCAN, 
                             KMeans,
                             AgglomerativeClustering)
from sklearn_extra.cluster import KMedoids
from sklearn.neighbors import NearestNeighbors
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import MissingIndicator, IterativeImputer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import (MinMaxScaler, 
                                   StandardScaler, 
                                   OneHotEncoder,
                                   KBinsDiscretizer,
                                   OrdinalEncoder)

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, f1_score
from xgboost import XGBClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from scipy.cluster.vq import whiten
from scipy.cluster.hierarchy import (dendrogram, 
                                     fcluster, 
                                     linkage,
                                     set_link_color_palette)

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from matplotlib.patches import RegularPolygon, Ellipse
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm, colorbar
from matplotlib import colors as mpl_colors

from matplotlib.lines import Line2D
import seaborn as sns

from matplotlib import __version__ as mplver
plt.rcParams['figure.dpi'] = 50

import warnings
from itertools import product
# from pandas_profiling import ProfileReport
import sqlite3
from math import ceil
#from yellowbrick.cluster import intercluster_distance
warnings.filterwarnings("ignore")

import plotly.io as pio
pio.templates.default = 'plotly_white'

<div class="alert alert-block alert-info">
<a class="anchor" id="data_understanding"> 
<h1>2. Data Understanding</h1>
</a>    
</div>

In [ ]:
# only for google colab
# comment the cell if you are running locally
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Loading the dataset and visualizing summary statistics
data_orig = pd.read_sas('a2z_insurance.sas7bdat',
                        encoding='unicode_escape', format='sas7bdat', index='CustID')

# location of file in google drive, file must be on the root folder of google drive
# data_orig = pd.read_sas('./drive/MyDrive/a2z_insurance.sas7bdat',
#                         encoding='unicode_escape', format='sas7bdat', index='CustID')


data = data_orig.copy()
data.index = data.index.astype('int64')
data.describe(include='all').T

In [ ]:
# Obtaining the shape of the dataset
data.shape

In [ ]:
# Show top rows
data.head()

In [ ]:
# check the data types
data.dtypes

In [ ]:
# Check missing values
data.isnull().sum()

There are 389 missing values in our dataset

From the DataFrame below, we can notice that in percentage, there is a small number of missing values and dropping them could be a viable option.

In [ ]:
#Checking missing in %
# this lines calculates the percentage of missing values
# but not the percentage of missing rows
percent_missing = data.isnull().sum() * 100 / len(data)
missing_values = pd.DataFrame({'name': data.columns,
                                 '%': percent_missing})
missing_values    

In [ ]:
# Check for duplicates
print('----------------------------\n'
      f'The dataset has {data.duplicated().sum()} duplicates\n'
      '----------------------------')

In [ ]:
data.drop_duplicates(inplace=True)

Checking the number of rows with `FirstPolYear` entrances being before the `BirthYear`

In [ ]:
dataTEMP = data[['FirstPolYear','BirthYear']]
dataTEMP = dataTEMP.fillna(0.0)
dataTEMP['DIF'] = np.where(data['FirstPolYear'] < data['BirthYear'], 1, 0)
print("------------------------\n"
      f"{dataTEMP['DIF'].sum()} iconsistent records\n"
      "------------------------")

We can see that 1997 registrations happened before the client was born.
This is impossible, therefore possible errors, or wrong entries either in the `BirthYear` or `FirstPolYear`.

In [ ]:
# Check types again
data.dtypes

<div class="alert alert-block alert-info">
<a class="anchor" id="data_Visualization"> 
<h1>3. Data Visualization</h1>
</a>    
</div>

In [ ]:
# function for scatter matrix

def scatter_matrix(df, cols=None, width=None, height=None, 
                   title='Scatter Matrix', color=None):
  fig = px.scatter_matrix(df, dimensions=cols,
                          width=width, height=height,
                          title=title,
                          color=color,
                          opacity=0.8,
                          color_continuous_scale='piyg')
  fig.layout.font.size = 7
  fig.update_traces(marker=dict(line=dict(color='white', width=0.7),
                                size=5))
  fig.update_layout(title_font_size=25)
  fig.update_xaxes(tickangle=90)
  fig.show()


In [ ]:
scatter_matrix(data.select_dtypes('float64'),
               width=1080, height=1080,
               title="Scatter matrix of initial numerical values")

We have only tree cathegorical variables: Education Degree, GeoLivArea and Children.
    
Therefore let's have a look and take some conclusions.

Below are count plots of the categorical features.

In [ ]:
def countplot(df, feature=None, ascending=False,
              title=None, labels=None, width=None,
              height=None, color=None):
    if feature == None:
        counts = df.value_counts(ascending=ascending)
    else:
        counts = df[feature].value_counts(ascending=ascending)

    plot = px.bar(counts, title=title,
                  labels=labels, width=width,
                  height=height)
    plot.update_layout(showlegend=False)
    plot.show()

In [ ]:
cat = data[['EducDeg', 'Children', 'GeoLivArea']]
cat['Children'] = cat['Children']\
    .apply(lambda x: 'Yes' if x == 1 else 'No')
cat = cat.dropna().astype('str')

In [ ]:
countplot(cat['EducDeg'], height=500, width=500,
          labels={'index': 'Education Degree',
                  'value': 'Frequency'})

<p>From the plot can be noticed that the "extremes" of education are less willing to subscribe the insurance (Basic degree and PhD).<br>The ones in the middle of education (BSc/MSc and High School) are more willing to subscribe.</p>


In [ ]:
countplot(cat['Children'], height=500, width=500,
          labels={'index': 'Children',
                  'value': 'Frequency'})

More people with children decide to register for the insurance

In [ ]:
# Comparing two cathegorical variables
data_counts = cat[['EducDeg', 'Children']]
data_counts = data_counts.groupby(['EducDeg', 'Children'],
                                  as_index=False).size()

educ_child_plot = px.bar(data_counts, x='EducDeg', y='size',
                         color=data_counts['Children'],
                         barmode='relative', title='Education vs Children',
                         labels={'size': 'Frequency', 
                                 'EducDeg': 'Education degree',
                                 'color': 'Children'},
                         height=500, width=500)

educ_child_plot.show()

Visually Children doesn't have high relation with Education, but we can't take conclusions through this plot, lets check later with a correlation matrix

In [ ]:
countplot(cat['GeoLivArea'], height=500, width=500,
          labels={'value': 'Frequency',
                  'index': 'Geographic Living Area'})

In [ ]:
#Exploratory Data Analysis (EDA) - Verifying Summary Statistics of numeric variables

# Select all numeric variables in the dataset
numerical = data.drop(['EducDeg', 'Children', 'GeoLivArea'],
                    axis=1).columns.tolist()

stats = ['skew', 'mad', 'kurt','mean', 'median', 'std', 'max', 'min', 'sum', 'count']
data[numerical].agg(stats).apply(lambda x: round(x, 2)).T

Trough a quick review, there are some unusual max and min values and there are also high skewness and high standar deviation in some variables, meaning a high dispersion of values.

In [ ]:
#HISTOGRAM OF ALL NUMERIC VARIABLES

# Draw
sns.set()
fig, ax = plt.subplots(2, 5, figsize=(15,7))
for var, subplot in zip(data[numerical], ax.flatten()):
    g = sns.histplot(data=data,
                bins=5,
                 x=var,
                 ax=subplot,
                 kde=False)

# Decoration
sns.despine()
plt.rc('axes')
fig.suptitle("Histograms with 5 bins of all numeric variables")
plt.show()


In [ ]:
# BOXPLOT (ALL NUMERIC VARIABLES)
# Draw
fig, ax = plt.subplots(4,3, figsize=(10,5), constrained_layout=True)
for var, subplot in zip(data[numerical], ax.flatten()):
    g = sns.boxplot(data=data,
                 x=var,ax=subplot)

# Decoration
sns.despine()
plt.rc('axes')
fig.suptitle("Boxplots of all numeric variables")
plt.show()


From the histograms and boxplots, outliers can be spotted. some of them have no meaning and may be consequence of bad inputs.

From the correlation matrix, a very strong linear correlation betwwen Customer Monthly Value and Claims rate can be seen. It's safe to drop one of the features to reduce dimensionality.

In [ ]:
# Create correlation matrix
corr = data[numerical].corr(method='pearson').apply(lambda x: round(x, 2))

corr_heatmap = px.imshow(corr, text_auto=True,
                         height=700, width=700,
                         title='Pearson correlation between numeric variables',
                         color_continuous_scale='rdbu')
corr_heatmap.show()

From the correlation matrix, a very strong linear correlation between Customer Monthly Value and Claims rate can be seen. It's safe to drop one of the features to reduce dimensionality.

In [ ]:
fig = px.scatter(data, x="ClaimsRate", y="CustMonVal",
                 height=500, width=700)
fig.update_traces(marker=dict(line=dict(color='white', width=0.7),
                              size=7))
fig.show()

In [ ]:
# SCATTER PLOT
# Verification of the negative correlation between  MonthSal and BirthYear

fig = px.scatter(data, y="MonthSal", x="BirthYear",
                 height=500, width=700)

fig.update_traces(marker=dict(line=dict(color='white', width=0.7),
                              size=7))
fig.update_xaxes(autorange=False, range=[1930, 2003])
fig.update_yaxes(autorange=False, range=[0, 5500])
fig.show()

### Exploration of categorical variables

__Cross Table__ EducDec and Children

In [ ]:
table1 = pd.crosstab(index=cat['EducDeg'], columns=cat['Children'])
table1

In [ ]:
# Visual Exploration of categorical variables of EducDeg and Children

fig = px.imshow(table1, text_auto=True,
                color_continuous_scale='rdbu',
                height=500, width=500)
fig.show()

In [ ]:
# Exploration of categorical variables: CROSS TABLE of EducDeg and GeoLivArea
table2 = pd.crosstab(index=cat['EducDeg'], columns=cat['GeoLivArea'])
table2

In [ ]:
fig = px.imshow(table2, text_auto=True,
                color_continuous_scale='rdbu',
                height=500, width=500)
fig.show()

<div class="alert alert-block alert-info">
<a class="anchor" id="data_reprocessing"> 
<h1>4. Data Preprocessing</h1>
</a>    
</div>

<div class="alert alert-block alert-success">
<a class="anchor" id="data_cleaning"> 
<h2>4.1. Data Cleaning</h2>
</a>    
</div>

### Outlier Detection and removal

In [ ]:
# Calculating the interquartile range for the numeric variable ClaimsRate
data_ = data['ClaimsRate']
q25, q75 = percentile(data_, 25), percentile(data_, 75)
iqr = q75 - q25
print(f'Percentiles: 25th={q25:.3f}, IQR={iqr:.3f}, 75th={q75:.3f}')

# calculate the outlier cutoff
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off

# identify outliers
outliers = [x for x in data_ if x < lower or x > upper]
print(f'Identified outliers: {len(outliers):}')

# Count outliers
outliers_removed = [x for x in data_ if x >= lower and x <= upper]
print(f'Non-outlier observations: {len(outliers_removed)}')

Let's perform this for all numeric variables

In [ ]:
numeric=['CustMonVal', 'FirstPolYear', 'BirthYear', 
         'MonthSal', 'PremLife', 'ClaimsRate', 'PremMotor', 
         'PremHousehold', 'PremHealth', 'PremWork']

for y in numeric:
    data_ = data[y]
    q25, q75 = percentile(data_, 25), percentile(data_, 75)
    iqr = q75 - q25
    print(f'Information for {y}\n'
          '-------------------------------------------------------------------------\n'
          f'Percentiles: 25th={q25:.3f}, IQR={iqr:.3f}, 75th={q75:.3f}')

# calculate the outlier cutoff
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off

# identify outliers
    outliers = [x for x in data_ if x < lower or x > upper]
    print(f'Identified outliers: {len(outliers):}')

# Count outliers
    outliers_removed = [x for x in data_ if x >= lower and x <= upper]
    print(f'Non-outlier observations: {len(outliers_removed)}\n'
          '-------------------------------------------------------------------------')

Creating a function to remove the outliers

In [ ]:
# Creating a Function to remove the outliers, based in the Interquartile range method
def remove_outliers(df, colList, lowPercentile=0.01, highPercentile=0.95, verbose=False):
    """Remove rows with outliers below or above the predefined percentile thresholds

    Args:
        df (dataframe): dataframe object
        colList (string list): list with names of columns
        lowPercentile (float, optional): Rows with value below this threshold will be removed. Defaults to 0.05.
        highPercentile (float, optional): Rows with value above this threshold will be removed. Defaults to 0.95.
        verbose (bool, optional): Indication if it should give feedback on the percentiles. Defaults to False.

    Returns:
        dataframe: the dataframe with the rows removed.
    """
    # Identify percentiles
    quant_df = df[colList].quantile([lowPercentile, highPercentile])
    if verbose:
        print(quant_df)

    # Loop in each column
    for name in colList:

        # Keep only rows that are inside the limits
        df = df[(df[name] >= quant_df.loc[lowPercentile, name]) & (df[name] <= quant_df.loc[highPercentile, name])]

    return df

In [ ]:
#Shape before outlier removal
dCP = data.copy(deep=True)
dCP_shape_before = dCP.shape
dCP_shape_before

In [ ]:
# Removing the outliers - different threshold for BirthYear - tried with low 0.00 and gave a very strange value

cols1=['CustMonVal', 'FirstPolYear', 'MonthSal', 'PremLife', 'ClaimsRate', 'PremMotor', 'PremHousehold', 'PremHealth', 'PremWork']
cols2 = ['BirthYear']

dCP = remove_outliers(dCP, colList=cols1 , lowPercentile=0.00, highPercentile=0.99, verbose=True)
dCP = remove_outliers(dCP, colList=cols2 , lowPercentile=0.01, highPercentile=0.99, verbose=True)
dCP_shape_after = dCP.shape
dCP_shape_after

In [ ]:
# Calculate percentage of data removed with outliers
PRemoved= (dCP_shape_before[0] - dCP_shape_after[0]) / dCP_shape_before[0]
print(f'Applying IQR method, we removed {round((PRemoved*100),2)}% of data')

In [ ]:
# refactoring the previous function into a class transformer

class RemoveOutliers(BaseEstimator, TransformerMixin):
    def __init__(self, cols, low=0.01, high=0.95, verbose=False):
        self.cols = cols
        self.low = low
        self.high = high
        self.verbose = verbose
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        df = X.copy()
        # Identify percentiles
        quant_df = df.quantile([self.low, self.high])
        if self.verbose:
            print(quant_df)
        

        for col in self.cols:
        # Keep only rows that are inside the limits
            df = df[(df[col] >= quant_df.loc[self.low, col])\
                    & (df[col] <= quant_df.loc[self.high, col])]

        return df
        

In [ ]:
# testing the transformer
pipe_removal = Pipeline([('rest', RemoveOutliers(cols1, 0.00, 0.99)),
                         ('birth', RemoveOutliers(cols2, 0.01, 0.99))])

# check if the function and transformer produce indentical results
pipe_removal.fit_transform(data).sum() == dCP.sum()

The DataFrames produced from the function and the class are identical.

#### Lets do some coherence checks to verify if the outliers removal was sucessfull:

In [ ]:
#Check first if there are observations above 120 years old or above the year of the data (2016)
Cohe_check1 = dCP.BirthYear.apply(lambda x: 1 if (x>2016 or x<1896) else 0)
Cohe_check1.sum()

There's no people above 120 years old or above the year of the data 2016 - outlier removal check sucessfull

In [ ]:
#Coherence check for Premiums (can't spend more money than they earn)
def summ(num1, *args):
    total=num1
    for num in args:
        total=total+num
    return total

Cohe_check2=dCP.apply(lambda x:1 if (summ(x.PremMotor, x.PremHousehold, x.PremLife, x.PremHealth, x.PremWork)>(x.MonthSal*12)) else 0, axis=1)
Cohe_check2.sum()

There's no people spending more money than they earn - outlier removal check sucessfull

In [ ]:
#Coherence check for Salary (legal age for working in Portugal is 16)
Cohe_check3=dCP.apply(lambda x:1 if (2016-x.BirthYear<16 and x.MonthlySal>0) else 0, axis=1)
Cohe_check3.sum()

All people working on this database are at least 16 years old - outlier removal check sucessfull

Another approach is to impute the outliers, next is a transfomer that fills the outliers with null values:

In [ ]:
# class that fills outliers with null
class FillOutliersNaN(BaseEstimator, TransformerMixin):
    def __init__(self, cols, lower=0.01, upper=0.95):
        self.cols = cols
        self.lower = lower
        self.upper = upper

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
    # Identify quantiles
        df = X.copy()
        # df['MonthSal'].mask(df['BirthYear'] > 1999, other=0.0, inplace=True)
        for col in self.cols:
            df[col] = df[col].mask(df[col] < df[col].quantile(self.lower))
            df[col] = df[col].mask(df[col] > df[col].quantile(self.upper))
        return df

In [ ]:
# Fill with 0 salary for people with 16 or less years old
class FillZero(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        new_X = X.copy()
        new_X = np.asarray(new_X)
        new_X[:,2] = np.where(new_X[:,9] > 1999.0, 0, new_X[:,2])
        return new_X


In [ ]:
cols1=['CustMonVal', 'FirstPolYear', 'MonthSal', 
       'PremLife', 'ClaimsRate', 'PremMotor', 
       'PremHousehold', 'PremHealth', 'PremWork']
cols2 = ['BirthYear']
cols3 = list(data.drop(cols1 + cols2, axis=1).columns)


In [ ]:
# output substitutes outliers with null values
# nul values can be dropped or imputed

ct = ColumnTransformer([('passthrough', 'passthrough', cols3),
                        ('birth_outliers', FillOutliersNaN(cols2, 0.01, 0.99), cols2),
                        ('rest_outliers', FillOutliersNaN(cols1, 0.00, 0.99), cols1)])

df = ct.fit_transform(data)
df = pd.DataFrame(df, index=data.index, 
                  columns=(cols3 + cols2 + cols1))

for feature in df.drop('EducDeg', axis=1).columns:
    df[feature] = df[feature].astype('float64')

df = df[data.columns]
df.head()


In [ ]:
corr = df.corr().apply(lambda x: round(x, 2))

corr_heatmap = px.imshow(corr, text_auto=True,
                         height=900, width=900,
                         title='Pearson correlation between numeric variables',
                         color_continuous_scale='rdbu')
corr_heatmap.show()

In [ ]:
df.shape

Pipeline removing outliers

In [ ]:
# pipeline for full imputation
num_pipe = Pipeline([
    ('fill_na', FillOutliersNaN(cols1, 0.01, 0.98)),
    ('fill_na_birth', FillOutliersNaN(cols2, 0.01, 0.99)),
    ('i2mputer', IterativeImputer()),
    ('fill_less_16', FillZero())
])

cat_pipe = Pipeline([('si', SimpleImputer(strategy='most_frequent'))])

imputer = ColumnTransformer([
    ('numbers', num_pipe, cols1 + cols2),
    ('categories', cat_pipe, cols3)
])

In [ ]:
df = imputer.fit_transform(data)
df = pd.DataFrame(df, index=data.index,
                  columns=(cols1 + cols2 + cols3))

for feature in df.drop('EducDeg', axis=1).columns:
    df[feature] = df[feature].astype('float64')

df = df[data.columns]
df.head()

In [ ]:
#Check first if there are observations above 120 years old or above the year of the data (2016)
df.BirthYear.apply(lambda x: 1 if (x>2016 or x<1896) else 0).sum()

In [ ]:
#Coherence check for Premiums (can't spend more money than they earn)
def summ(num1, *args):
    total=num1
    for num in args:
        total=total+num
    return total

df.apply(lambda x:1 if (summ(x.PremMotor, x.PremHousehold, 
                                      x.PremLife, x.PremHealth, 
                                      x.PremWork)>(x.MonthSal*12)) else 0, axis=1).sum()


In [ ]:
#Coherence check for Salary (legal age for working in Portugal is 16)
df.apply(lambda x:1 if (2016-x.BirthYear<16\
                        and x.MonthSal>0) else 0, axis=1).sum()

In [ ]:
corr = df.corr().apply(lambda x: round(x, 2))

corr_heatmap = px.imshow(corr, text_auto=True,
                         height=900, width=900,
                         title='Pearson correlation between numeric variables',
                         color_continuous_scale='rdbu')
corr_heatmap.show()

Due interpolation, 10 obervations matches to people with less than 16 years old and monthly salary above 0. This observations can be imputed with 0

<div class="alert alert-block alert-success">
<a class="anchor" id="features_engineering"> 
<h2>4.2. Features Engineering</h2>
</a>    
</div>


### Derived Attributes

In [ ]:
#Adding Age
dCP['Age'] = 2016 - dCP['BirthYear']
dCP['Age'] = dCP['Age']

In [ ]:
#Loyalty - time as a client
dCP['Loyalty'] = 2016 - dCP['FirstPolYear']
dCP['Loyalty'] = dCP['Loyalty']

As premiums are already in euros per year, makes sense to convert all values to annualy

In [ ]:
#Client annual salary
dCP['AnualSal'] = dCP['MonthSal'] * 12.0


In [ ]:
# calculation of total premiums
dCP['total_premium'] = dCP['PremMotor'] + dCP['PremHousehold']\
+ dCP['PremHealth'] + dCP['PremLife'] + dCP['PremWork']

In [ ]:
# Adding Positive and negative premiums
# dCP['PositivePrem'] = np.where(dCP['total_premium'] > 0, dCP['total_premium'], 0)
# dCP ['NegativePrem'] = np.where(dCP ['total_premium'] < 0, dCP ['total_premium'], 0)

In [ ]:
# Adding Positive the Effort_Ratio
dCP['Effort_Ratio'] = dCP['total_premium'] / dCP['AnualSal']

In [ ]:
# Insurance types ratio in tota premiums

dCP['PremMotorRatio'] = dCP['PremMotor'] / dCP['total_premium']
dCP['PremHouseholdRatio'] = dCP['PremHousehold'] / dCP['total_premium']
dCP['PremHealthRatio'] = dCP ['PremHealth'] / dCP['total_premium']
dCP['PremLifeRatio'] = dCP['PremLife'] / dCP['total_premium']
dCP['PremWorkRatio'] = dCP['PremWork'] / dCP['total_premium']


In [ ]:
#Does the customer represent a profit for the company or not?
#If ClaimsRate is below 1, the client represents Profit to the company)
dCP['Profit'] = 0
dCP['Profit'] = dCP['Profit'].where(dCP['ClaimsRate']>1,1)

In [ ]:
dCP.head()

Make a transformer to create the previous attributes

In [ ]:
class CreateFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
 
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        df = X.copy()
        
        # creates age
        df['Age'] = (2016.0 - df['BirthYear'])
        
        # creates loyalty
        df['Loyalty'] = (2016.0 - df['FirstPolYear'])
        
        # creates annual salary
        df['AnualSal'] = df['MonthSal'] * 12
        
        # calculates total premiums
        df['total_premium'] = 0
        for premium in df.columns:
            if 'Prem' in premium:
                df['total_premium'] += df[premium]

        
        # Adding Positive the Effort_Ratio
        df['Effort_Rate'] = df['total_premium'] / df['AnualSal']
        
        # Insurance types ratio in tota premiums
        for premium in df.columns:
            if premium == f'{premium}Ratio':
                break
            if 'Prem' in premium:
                df[f'{premium}Ratio'] = df[premium] / df['total_premium']

        
        # profit binary variable
        df['Profit'] = 0
        df['Profit'] = df['Profit'].where(df['ClaimsRate'] > 1,1)
        
        return df

In [ ]:
# test the transformer

cf = CreateFeatures()
df2 = cf.fit_transform(df)
df2.head()

In [ ]:
# add miningfull names to variable features list
birth_year = cols2
rest_num_feats = cols1
num_feats = cols1 + cols2
cat_feats = cols3

Feature Creation must go after KNNImputer in pipeline, KNNImputer output is an array an the new class for Feature creation needs a DataFrame as input, the following class is a refactoring to accept np.arrays

In [ ]:
class CreateFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features
 
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        df = X.copy()
        
        # creates age
        df = pd.DataFrame(df, columns=self.features)
        df['Age'] = (2016.0 - df['BirthYear'])
        
        # creates loyalty
        df['Loyalty'] = (2016.0 - df['FirstPolYear'])
        
        # creates annual salary
        df['AnualSal'] = df['MonthSal'] * 12
        
        # calculates total premiums
        df['total_premium'] = 0
        for premium in df.columns:
            if 'Prem' in premium:
                df['total_premium'] += df[premium]

        
        # Adding Positive the Effort_Ratio
        df['Effort_Rate'] = df['total_premium'] / df['AnualSal']
        
        # Insurance types ratio in tota premiums
        for premium in df.columns:
            if premium == f'{premium}Ratio':
                break
            if 'Prem' in premium:
                df[f'{premium}Ratio'] = df[premium] / df['total_premium']

        
        # profit binary variable
        df['Profit'] = 0
        df['Profit'] = df['Profit'].where(df['ClaimsRate'] > 1,1)
        
        return df

In [ ]:
new_feats = ['Age', 'Loyalty', 'AnualSal', 
 'total_premium', 'Effort_Rate', 
 'PremMotorRatio', 'PremHouseholdRatio',
 'PremHealthRatio', 'PremLifeRatio',
 'PremWorkRatio', 'Profit']

In [ ]:
# Add the feature engineering to the pipeline
num_pipe = Pipeline([
    ('fill_na', FillOutliersNaN(cols1, 0.01, 0.98)),
    ('fill_na_birth', FillOutliersNaN(cols2, 0.01, 0.99)),
    ('i2mputer', IterativeImputer()),
    ('fill_less_16', FillZero()),
    ('feat_engineering', CreateFeatures(num_feats))
])

cat_pipe = Pipeline([('si', SimpleImputer(strategy='most_frequent'))])

feature_creator = ColumnTransformer([
    ('numbers', num_pipe, num_feats),
    ('categories', cat_pipe, cat_feats)
])

In [ ]:
output_data = feature_creator.fit_transform(data)
df = pd.DataFrame(output_data, index=data.index, 
                  columns=(num_feats + new_feats + cat_feats))

for feature in df.drop('EducDeg', axis=1).columns:
    df[feature] = df[feature].astype('float64')
df = df[list(data.columns) + new_feats]
df.head()

### Single-attributes transformation

In [ ]:
df.describe(include='all').T

In [ ]:
corr = df.corr().apply(lambda x: round(x, 2))

corr_heatmap = px.imshow(corr, text_auto=True,
                         height=900, width=900,
                         title='Pearson correlation between numeric variables',
                         color_continuous_scale='rdbu')
corr_heatmap.show()

There is high correlation between features because new features were engineered using existing features. Some of them will be dropped or transformed into bins.

<div class="alert alert-block alert-success">
<a class="anchor" id="data_scaling"> 
<h2>4.3. Data Scaling</h2>
</a>    
</div>

Data must be scale in roder to be comparable in distances.

In [ ]:
# add standard scaler to the pipeline
num_pipe = Pipeline([
    ('fill_na', FillOutliersNaN(rest_num_feats, 0.01, 0.99)),
    ('fill_na_birth', FillOutliersNaN(birth_year, 0.01, 0.99)),
    ('knn_imputer', KNNImputer()),
    ('feat_engineering', CreateFeatures(num_feats)),
    ('scaling', MinMaxScaler())
])

cat_pipe = Pipeline([('si', SimpleImputer(strategy='most_frequent')),
                     ('ordinal_ecoder', OrdinalEncoder()),
                     ('scaling', MinMaxScaler())])

pipeline = ColumnTransformer([
    ('numbers', num_pipe, num_feats),
    ('categories', cat_pipe, cat_feats)
])

In [ ]:
output_data = pipeline.fit_transform(data)
df = pd.DataFrame(output_data, index=data.index, 
                  columns=(num_feats + new_feats + cat_feats))

for feature in df.drop('EducDeg', axis=1).columns:
    df[feature] = df[feature].astype('float64')
df = df[list(data.columns) + new_feats]
df.head()

In [ ]:
# transform dtypes of categorical to object
df[cat_feats + ['Profit']] = df[cat_feats + ['Profit']].astype('object')
df.info()

In [ ]:
sns.set()

fig, axes = plt.subplots(6, 3, figsize=(20, 25))

for ax, feat in zip(axes.flatten(), df.select_dtypes('float64')):
    sns.boxplot(x = df["GeoLivArea"].apply(lambda x: round(x, 2)), 
                y = df[feat], ax = ax)
    
plt.show()

From the correlation heatmap graph, GeoLive Area seemed to have low to none correlation with any of the variables which can be confirmed in the boxplots.

<div class="alert alert-block alert-success">
<a class="anchor" id="feature_selection"> 
<h2>4.4. Feature Selection</h2>
</a>    
</div>

<div class="alert alert-block alert-warning">
<a class="anchor" id="imports"> 
<h3>Principal Components Analysis (PCA)</h3>
</a> 
<p>In the context of this project PCA is used to find important features in our dataset.</p>
</div>



In [ ]:
# excluded age due high correlation with AnualSal
metric_features = ['Loyalty', 'Age',
                   'total_premium', 'Effort_Rate',
                   'CustMonVal', 'PremMotorRatio', 
                   'PremHouseholdRatio', 'PremHealthRatio', 
                   'PremLifeRatio', 'PremWorkRatio']

cat_features = ['Children', 'EducDeg', 'Profit']

# features insurance consumption
insurances = ['PremMotorRatio', 'PremHouseholdRatio',
              'PremHealthRatio', 'PremLifeRatio',
              'PremWorkRatio']

# socio-demographic metric features
sdemo = ['EducDeg', 'Children', 'AnualSal']

# monetary value feature
valuef = ['CustMonVal', 'Effort_Rate', 'total_premium']
        

In [ ]:
pca = PCA()
data_pca = pca.fit_transform(df[metric_features])

In [ ]:
pd.DataFrame(
    {"Eigenvalue": pca.explained_variance_,
     "Difference": np.insert(np.diff(pca.explained_variance_), 0, 0),
     "Proportion": pca.explained_variance_ratio_,
     "Cumulative": np.cumsum(pca.explained_variance_ratio_)},
    index=range(1, pca.n_components_ + 1)
)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(shared_xaxes=True, shared_yaxes=True)
                   
fig.add_trace(
    go.Scatter(x=[i for i in range(1, 11)], 
               y=pca.explained_variance_ratio_,
               name='Cumulative'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=[i for i in range(1, 11)], 
               y=np.cumsum(pca.explained_variance_ratio_),
               name='Proportion'),
    row=1, col=1
)

fig.update_layout(autosize=False, height=500, width=700, 
                  title_text='Variance Explained')
fig.show()

In [ ]:
pca = PCA(n_components=5, random_state=42)
data_pca = pca.fit_transform(df[metric_features])

5 components explain 93% of the variance

In [ ]:
pd.DataFrame(abs(pca.components_))

In [ ]:
df_pca = pd.DataFrame(abs(pca.components_),
                      columns=metric_features, 
                      index=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])\
                      .apply(lambda x: round(x, 2))
fig = px.imshow(df_pca, text_auto=True,
                title='Feature importances by PCA',
                color_continuous_scale='inferno',
                height=500, width=800)

fig.show()

From the heatmap, can be noticed that the features that explain the majority of the variance are `Loyalty`, `total_premium`, `Age`, `CustMonVal`, `PremHouselHoldRatio`, `PremHealthRatio` and `PremLifeRatio`.

In [ ]:
best_feats = ['Loyalty', 'Age', 'total_premium', 
              'CustMonVal', 'PremHouseholdRatio', 
              'PremHealthRatio', 'PremLifeRatio']

### T-SNE

The data is transformed with the TSNE class for later visualizations.

In [ ]:
tsne = TSNE(random_state=42, n_jobs=-1)
tsne_data = tsne.fit_transform(df[best_feats])

<div class="alert alert-block alert-info">
<a class="anchor" id="clustering"> 
<h1>5. Clustering</h1>
</a>    
</div>

<div class="alert alert-block alert-success">
<a class="anchor" id="dbscan"> 
<h2>5.1. DBSCAN</h2>
</a>    
</div>

Using the pca features found in the above cell

In [ ]:
# finding optimal eps value for DBSCAN

nn = NearestNeighbors(n_neighbors=50, algorithm='brute', n_jobs=-1)
nn.fit(df[best_feats])
distances, indices = nn.kneighbors(df[best_feats])

# sort distances
distances = np.sort(distances[:,1], axis=0)

fig = px.line(distances)
fig.update_layout(autosize=False,
                  height=500, width=700)
fig.show()

In [ ]:
min_samples = range(8, 21, 2)
eps = np.arange(0.15, 0.25, 0.01)
# eps = np.arange(0.18, 0.25, 0.01)

output = []
for ms in min_samples:
  for ep in eps:
    labels = DBSCAN(min_samples=ms, eps = ep,
                    n_jobs=-1).fit(df[best_feats]).labels_
    score = silhouette_score(df[best_feats], labels)
    output.append((ms, ep, score))

In [ ]:
# Get the parameters for best silhouette score
min_samples, eps, score = sorted(output, key=lambda x:x[-1])[-1]
print(f'Best silhouette_score: {score}')
print(f'min_samples: {min_samples}')
print(f'eps: {eps}')

In [ ]:
# Get the number of clusters and outliers for DBSCAN with best parameters
labels_dbscan = DBSCAN(min_samples=min_samples, 
                       eps=eps, n_jobs=-1).fit(df[best_feats]).labels_
clusters = len(Counter(labels_dbscan))
print(f"Number of clusters: {clusters}")
print(f"Number of outliers: {Counter(labels_dbscan)[-1]}")
print(f"Silhouette_score: {silhouette_score(df[best_feats], labels_dbscan)}")

From DBSCAN it can be noticed that there are few outliers and compared to the number of observations (10293) their quantity carry little weight.

In [ ]:
labels_str_dbscan = labels_dbscan.astype('str')

In [ ]:
scatter_matrix(df, best_feats, color=labels_str_dbscan, 
               width=800, height=800,
               title='Scatter matrix of metric features, DBSCAN')

In [ ]:
fig = px.scatter(x=data_pca[:, 0], y=data_pca[:, 1],
                 color=labels_str_dbscan)
fig.update_traces(
    marker=dict(line=dict(color='white', width=0.7),
                size=7)
)
fig.update_layout(
    xaxis_title='PC 1',
    yaxis_title='PC 2',
    legend_title='Cluster',
    title='Clusters of data by PC',
    autosize=False,
    height=500, width=500
)
fig.show()

In [ ]:
fig = px.scatter_3d(x=data_pca[:, 0], y=data_pca[:, 1],
                    z=data_pca[:, 2], width=800,
                    color=labels_str_dbscan, height=700,
                    labels={'x': 'PC 1', 'y':'PC 2', 'z':'PC 3'})
fig.update_traces(
    marker=dict(line=dict(color='white', width=0.7),
                size=5)
)
fig.update_layout(
    legend_title='Cluster',
    title='Clusters of data by PC',
    margin=dict(
    l=40, r=40, b=100, t=100, pad=4
    )
)
fig.show()

In [ ]:

data_imputed = imputer.fit_transform(data)
data_imputed = pd.DataFrame(data_imputed, index=data.index,
                            columns=cols1 + cols2 + cols3)
data_imputed[cols1 + cols2] = data_imputed[cols1 + cols2].astype('float64')
data_imputed = data_imputed[data.columns]
data_imputed['Labels_DBSCAN'] = labels_dbscan

In [ ]:
# function for plotting boxplots of original features with newly found labels

def boxplot_labels(dataframe, labels):
    features = data_imputed.select_dtypes('float64').columns
    fig = make_subplots(rows=5, cols=2,
                       subplot_titles=features)

    fig.add_trace(
        go.Box(x=data_imputed[labels],
               y=data_imputed['FirstPolYear'],
               name='First Policy Year'),
        row=1, col=1
    )
    fig.add_trace(
        go.Box(x=data_imputed[labels],
               y=data_imputed['BirthYear'],
               name='Birth Year'),
        row=1, col=2
    )
    fig.add_trace(
        go.Box(x=data_imputed[labels],
               y=data_imputed['MonthSal'],
               name='Monthly Salary'),
        row=2, col=1
    )
    fig.add_trace(
        go.Box(x=data_imputed[labels],
               y=data_imputed['CustMonVal'],
               name='Customer Monetary Value'),
        row=2, col=2
    )

    fig.add_trace(
        go.Box(x=data_imputed[labels],
               y=data_imputed['ClaimsRate'],
               name='Claims Rate'),
        row=3, col=1
    )

    fig.add_trace(
        go.Box(x=data_imputed[labels],
               y=data_imputed['PremMotor'],
               name='Premium Motor'),
        row=3, col=2
    )

    fig.add_trace(
        go.Box(x=data_imputed[labels],
               y=data_imputed['PremHousehold'],
               name='Premium Household'),
        row=4, col=1
    )

    fig.add_trace(
        go.Box(x=data_imputed[labels],
               y=data_imputed['PremHealth'],
               name='Premium Health'),
        row=4, col=2
    )

    fig.add_trace(
        go.Box(x=data_imputed[labels],
               y=data_imputed['PremLife'],
               name='Premium Life'),
        row=5, col=1
    )

    fig.add_trace(
        go.Box(x=data_imputed[labels],
               y=data_imputed['PremWork'],
               name='Premium Work'),
        row=5, col=2
    )

    fig.update_layout(height=1600, width=1000, 
                      title_text="Stacked Subplots",
                      legend_title='Feature')
    fig.show()

In [ ]:
boxplot_labels(data_imputed, 'Labels_DBSCAN')

We cannot draw important conclusion from de clusters acquired through DBSCAN.

<div class="alert alert-block alert-success">
<a class="anchor" id="kmeans"> 
<h2>5.2. KMeans</h2>
</a>    
</div>

In [ ]:
# applying Kmeans to the same variables
output = []
scores = []
inertias = []
k_vals = range(2,10) 
for k in k_vals:
  km = KMeans(n_clusters=k, random_state=42).fit(df[best_feats])
  labels = km.labels_
  score = silhouette_score(df[best_feats], labels)
  inertia = km.inertia_
  output.append((k, score))
  scores.append(score)
  inertias.append(inertia)

In [ ]:
k, score = sorted(output, key=lambda x:x[-1])[-1]
fig = px.line(x=k_vals, y=scores, height=500, width=700,
              labels={'x':'k', 'y':'Silhouette score'})
fig.show()
print(f"Best silhouette_score: {score}")
print(f"k: {k}")

In [ ]:
fig = px.line(x=k_vals, y=inertias, height=500, width=700,
              labels={'x':'k', 'y':'Inertia'})
fig.show()

Considering that the best silhoute score does not differ too much for other clusters sizes(2 -> 0.230, 3 -> 0.227) , we can choose 3 clusters to match the elbow method.

In [ ]:
k = 3
km = KMeans(n_clusters=k, random_state=42).fit(df[best_feats])
distances_kmeans = km.transform(df[best_feats])
labels_kmeans = km.labels_

In [ ]:
labels_str_kmeans = labels_kmeans.astype('str')
scatter_matrix(df, best_feats, color=labels_str_kmeans, 
               width=800, height=800,
               title='Scatter matrix of metric features, KMeans Clusters')

In [ ]:
# plot clusters cardinality

freq_cluster = df.groupby(labels_kmeans).size()
fig = px.bar(x=range(k), y=freq_cluster,
             title='Cluster cardinality', color=[str(i) for i in range(k)])
fig.update_xaxes(type='category')
fig.update_layout(
    xaxis_title='Cluster',
    yaxis_title='Sum of distances to centroid',
    legend_title='Cluster',
    width=500, height=500
)
fig.show()


In [ ]:
# Plot clusters magnitude
df['distanceToCentroid'] = np.min(distances_kmeans, axis=1)
magnitud = df.groupby(labels_kmeans).sum()['distanceToCentroid']
df.drop('distanceToCentroid', axis=1, inplace=True)
fig = px.bar(x=magnitud.index, y=magnitud.values,
             color=[str(i) for i in range(k)])
fig.update_xaxes(type='category')
fig.update_layout(
    xaxis_title='Cluster',
    yaxis_title='Sum of distances to centroid',
    legend_title='Cluster',
    width=600, height=500
)
fig.show()

Cluster magnitude is the sum of distances from all examples to the centroid of the cluster.

In [ ]:
# plot cardinality vs magnitud

fig = px.scatter(x=freq_cluster, y=magnitud, trendline='ols',
                 title='Magnitud vs. Cardinality')
fig.update_layout(
    xaxis_title='Cardinality',
    yaxis_title='Magnitud',
    width=700,
    height=500
)
fig.show()
results = px.get_trendline_results(fig)
results.px_fit_results.iloc[0].summary()

Notice that a higher cluster cardinality tends to result in a higher cluster magnitude, which intuitively makes sense. Clusters are anomalous when cardinality doesn't correlate with magnitude relative to the other clusters.

Next are are scatter plots of the principal components obtained in the PCA but color coded with the cluster obtained using non-PCA features.

In [ ]:
fig = px.scatter(x=data_pca[:, 0], y=data_pca[:, 1],
                 color=labels_str_kmeans)
fig.update_traces(
    marker=dict(line=dict(color='white', width=0.7),
                size=7)
)
fig.update_layout(
    xaxis_title='PC 1',
    yaxis_title='PC 2',
    legend_title='Cluster',
    title='Clusters of data by PC',
    height=500,
    width=500
)
fig.show()

In [ ]:
fig = px.scatter(x=data_pca[:, 0], y=data_pca[:, 2],
                 color=labels_str_kmeans)
fig.update_traces(
    marker=dict(line=dict(color='white', width=0.7),
                size=7)
)
fig.update_layout(
    xaxis_title='PC 1',
    yaxis_title='PC 3',
    legend_title='Cluster',
    title='Clusters of data by PC',
    height=500,
    width=500
)
fig.show()

In [ ]:
fig = px.scatter_3d(x=data_pca[:, 0], y=data_pca[:, 1],
                    z=data_pca[:, 2], width=700,
                    color=labels_str_kmeans, height=700,
                    labels={'x': 'PC 1', 'y':'PC 2', 'z':'PC 3'})
fig.update_traces(
    marker=dict(line=dict(color='white', width=0.7),
                size=5)
)
fig.update_layout(
    legend_title='Cluster',
    title='Clusters of data by PC',
    margin=dict(
    l=40, r=40, b=100, t=100, pad=4
    )
)
fig.show()

In [ ]:
fig = px.scatter(x=tsne_data[:, 0], y=tsne_data[:, 1],
                 color=labels_str_kmeans, opacity=0.8)
fig.update_traces(
    marker=dict(line=dict(color='white', width=0.2),
                size=4)
)
fig.update_layout(
    xaxis_title='First Component',
    yaxis_title='Second Component',
    legend_title='Cluster',
    title='t-sne visualization',
    height=500,
    width=500
)
fig.show()

In [ ]:
data_imputed['Labels_KMeans'] = labels_kmeans

In [ ]:
boxplot_labels(data_imputed, 'Labels_KMeans')

Observations seem to be clustered around age/salary with a different tendency in consumption.

In [ ]:
# Comparing two cathegorical variables
data_clustered = data_imputed[['EducDeg', 'Labels_KMeans']]
data_clustered = data_clustered.groupby(['EducDeg', 'Labels_KMeans'],
                                  as_index=False).size()

fig = px.bar(data_clustered, x='EducDeg', y='size',
             color=data_clustered['Labels_KMeans'].astype('str'),
             barmode='relative', title='Education degree by cluster',
             labels={'size': 'Frequency', 
                     'EducDeg': 'Education degree',
                     'color': 'Cluster'},
             height=500, width=500)

fig.show()

**Cluster 2** tends to be of people with higher education.

People with basic education tend to be in **cluster 0** (youngest), nevertheless the cluster is composed of people with different levels of education in similar frequency until BSc/MSc.

People from **cluster 1** (the oldest) tend to be more educated between highschool and Bachelor's/Master's.

In [ ]:
# Comparing two cathegorical variables
data_clustered = data_imputed[['Children', 'Labels_KMeans']]
data_clustered = data_clustered.groupby(['Children', 'Labels_KMeans'],
                                  as_index=False).size()

fig = px.bar(data_clustered, x='Children', y='size',
             color=data_clustered['Labels_KMeans'].astype('str'),
             barmode='relative', title='Children by cluster',
             labels={'size': 'Frequency',
                     'color': 'Cluster'},
             height=500, width=500)

fig.show()

Most people without children belong to **cluster 1**, while in **clusters 0 and 2** tend to have children.

<div class="alert alert-block alert-success">
<a class="anchor" id="kmedoids"> 
<h2>5.3. KMedoids</h2>
</a>    
</div>

In [ ]:
inertias = []
k_vals = range(1, 20)
for k in k_vals:
    kmed = KMedoids(n_clusters=k, random_state=42)
    kmed.fit(df[best_feats])
    inertias.append(kmed.inertia_)

In [ ]:
fig = px.line(x=k_vals, y=inertias, height=500, width=700,
              labels={'x':'k', 'y':'Inertia'})
fig.show()

In [ ]:
k = 3
kmedoids = KMedoids(n_clusters=k, random_state=42)
distances_kmedoids = kmedoids.fit_transform(df[best_feats])
labels_kmedoids = kmedoids.labels_

In [ ]:
labels_str_kmedoids = labels_kmedoids.astype('str')
scatter_matrix(df, best_feats, color=labels_str_kmedoids, 
               width=800, height=800,
               title='Scatter matrix of metric features, KMedoids Clusters')

In [ ]:
# plot clusters cardinality

freq_cluster = df.groupby(labels_kmedoids).size()
fig = px.bar(x=range(k), y=freq_cluster,
             title='Cluster cardinality', color=[str(i) for i in range(k)])
fig.update_xaxes(type='category')
fig.update_layout(
    xaxis_title='Cluster',
    yaxis_title='Sum of distances to centroid',
    legend_title='Cluster',
    width=500, height=500
)
fig.show()


In [ ]:
# Plot clusters magnitude
df['distanceToCentroid'] = np.min(distances_kmedoids, axis=1)
magnitud = df.groupby(labels_kmedoids).sum()['distanceToCentroid']
df.drop('distanceToCentroid', axis=1, inplace=True)
fig = px.bar(x=magnitud.index, y=magnitud.values,
             color=[str(i) for i in range(k)])
fig.update_xaxes(type='category')
fig.update_layout(
    xaxis_title='Cluster',
    yaxis_title='Sum of distances to centroid',
    legend_title='Cluster',
    width=500, height=500
)
fig.show()

Cluster magnitude is the sum of distances from all examples to the centroid of the cluster.

In [ ]:
# plot cardinality vs magnitud

fig = px.scatter(x=freq_cluster, y=magnitud, trendline='ols',
                 title='Magnitud vs. Cardinality')
fig.update_layout(
    xaxis_title='Cardinality',
    yaxis_title='Magnitud',
    width=700,
    height=500
)
fig.show()
results = px.get_trendline_results(fig)
results.px_fit_results.iloc[0].summary()

Notice that a higher cluster cardinality tends to result in a higher cluster magnitude, which intuitively makes sense. Clusters are anomalous when cardinality doesn't correlate with magnitude relative to the other clusters.

In [ ]:
fig = px.scatter(x=data_pca[:, 0], y=data_pca[:, 1],
                 color=labels_str_kmedoids)
fig.update_traces(
    marker=dict(line=dict(color='white', width=0.7),
                size=7)
)
fig.update_layout(
    xaxis_title='PC 1',
    yaxis_title='PC 2',
    legend_title='Cluster',
    title='Clusters of data by PC',
    height=500,
    width=500
)
fig.show()

In [ ]:
fig = px.scatter_3d(x=data_pca[:, 0], y=data_pca[:, 1],
                    z=data_pca[:, 2], width=700,
                    color=labels_str_kmedoids, height=700,
                    labels={'x': 'PC 1', 'y':'PC 2', 'z':'PC 3'})
fig.update_traces(
    marker=dict(line=dict(color='white', width=0.7),
                size=5)
)
fig.update_layout(
    legend_title='Cluster',
    title='Clusters of data by PC',
    margin=dict(
    l=40, r=40, b=100, t=100, pad=4
    )
)
fig.show()

In [ ]:
fig = px.scatter(x=tsne_data[:, 0], y=tsne_data[:, 1],
                 color=labels_str_kmedoids, opacity=0.8)
fig.update_traces(
    marker=dict(line=dict(color='white', width=0.2),
                size=4)
)
fig.update_layout(
    xaxis_title='First Component',
    yaxis_title='Second Component',
    legend_title='Cluster',
    title='t-sne visualization',
    height=500,
    width=500
)
fig.show()

In [ ]:
data_imputed['Labels_kmedoids'] = labels_kmedoids

In [ ]:
boxplot_labels(data_imputed, 'Labels_kmedoids')

In [ ]:
# Comparing two cathegorical variables
data_clustered = data_imputed[['EducDeg', 'Labels_kmedoids']]
data_clustered = data_clustered.groupby(['EducDeg', 'Labels_kmedoids'],
                                  as_index=False).size()

fig = px.bar(data_clustered, x='EducDeg', y='size',
             color=data_clustered['Labels_kmedoids'].astype('str'),
             barmode='relative', title='Education degree by cluster',
             labels={'size': 'Frequency', 
                     'EducDeg': 'Education degree',
                     'color': 'Cluster'},
             height=500, width=500)

fig.show()

In [ ]:
# Comparing two cathegorical variables
data_clustered = data_imputed[['Children', 'Labels_kmedoids']]
data_clustered = data_clustered.groupby(['Children', 'Labels_kmedoids'],
                                  as_index=False).size()

fig = px.bar(data_clustered, x='Children', y='size',
             color=data_clustered['Labels_kmedoids'].astype('str'),
             barmode='relative', title='Children by cluster',
             labels={'size': 'Frequency',
                     'color': 'Cluster'},
             height=500, width=500)

fig.show()

<div class="alert alert-block alert-success">
<a class="anchor" id="hclustering"> 
<h2>5.4. Hierarchical Clustering</h2>
</a>    
</div>


In [ ]:
h_cluster = linkage(df[best_feats], method='ward', metric='euclidean')

In [ ]:
fig = plt.figure(figsize=(12,7))
dendrogram(h_cluster, color_threshold=20, 
           orientation='top', no_labels=True, 
           above_threshold_color='k')

plt.hlines(20, 0, 100000, colors="r", linestyles='dashed')
plt.title("Hierarchical Clustering - Ward's Dendrogram", fontsize=23)
plt.ylabel('Euclidean Distance', fontsize=13)
plt.show()

In [ ]:
h_clustering = AgglomerativeClustering(n_clusters=3, affinity='euclidean', 
                                       linkage='ward')
h_clustering.fit(df[best_feats])
labels_hcluster = h_clustering.labels_

In [ ]:
labels_str_hcluster = labels_hcluster.astype('str')
scatter_matrix(df, best_feats, color=labels_str_hcluster, 
               width=800, height=800,
               title='Scatter matrix of metric features, Heirarchical Clusters')

In [ ]:
fig = px.scatter(x=data_pca[:, 0], y=data_pca[:, 1],
                 color=labels_str_hcluster, opacity=0.8)
fig.update_traces(
    marker=dict(line=dict(color='white', width=0.7),
                size=7)
)
fig.update_layout(
    xaxis_title='PC 1',
    yaxis_title='PC 2',
    legend_title='Cluster',
    title='Clusters of data by PC',
    height=500,
    width=500
)
fig.show()

In [ ]:
fig = px.scatter_3d(x=data_pca[:, 0], y=data_pca[:, 1],
                    z=data_pca[:, 2], width=700,
                    color=labels_str_hcluster, height=700,
                    labels={'x': 'PC 1', 'y':'PC 2', 'z':'PC 3'})
fig.update_traces(
    marker=dict(line=dict(color='white', width=0.7),
                size=5)
)
fig.update_layout(
    legend_title='Cluster',
    title='Clusters of data by PC',
    margin=dict(
    l=40, r=40, b=100, t=100, pad=4
    )
)
fig.show()

In [ ]:
fig = px.scatter(x=tsne_data[:, 0], y=tsne_data[:, 1],
                 color=labels_str_hcluster, opacity=0.8)
fig.update_traces(
    marker=dict(line=dict(color='white', width=0.2),
                size=4)
)
fig.update_layout(
    xaxis_title='First Component',
    yaxis_title='Second Component',
    legend_title='Cluster',
    title='t-sne visualization',
    height=500,
    width=500
)
fig.show()

In [ ]:
data_imputed['Labels_hcluster'] = labels_hcluster

In [ ]:
boxplot_labels(data_imputed, 'Labels_hcluster')

In [ ]:
# Comparing two cathegorical variables
data_clustered = data_imputed[['EducDeg', 'Labels_hcluster']]
data_clustered = data_clustered.groupby(['EducDeg', 'Labels_hcluster'],
                                  as_index=False).size()

fig = px.bar(data_clustered, x='EducDeg', y='size',
             color=data_clustered['Labels_hcluster'].astype('str'),
             barmode='relative', title='Education degree by cluster',
             labels={'size': 'Frequency', 
                     'EducDeg': 'Education degree',
                     'color': 'Cluster'},
             height=500, width=500)

fig.show()

In [ ]:
# Comparing two cathegorical variables
data_clustered = data_imputed[['Children', 'Labels_hcluster']]
data_clustered = data_clustered.groupby(['Children', 'Labels_hcluster'],
                                  as_index=False).size()

fig = px.bar(data_clustered, x='Children', y='size',
             color=data_clustered['Labels_hcluster'].astype('str'),
             barmode='relative', title='Education degree by cluster',
             labels={'size': 'Frequency',
                     'color': 'Cluster'},
             height=500, width=500)

fig.show()

## Initial conlusions

Except for DBSCAN, the three algorithms segment the clients in a similar fashion, mainly by age.

We have three groups wich can be called Senior, Middle, and Junior.

KMeans showed better differentiation in the boxplots.

**Middle age group**: tend to have lower anual premiums in work, life, Health, and Household and higher premiums in motor insurances.

**Junior age group**: tend to have in average higher Anual Premiums in work, life, Health, and Household and lower in motor premium insurances.

**Senior age group**: tend to have in average, an intermediate value in anual premiums in all categories.

In [ ]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = f'{n}%'
    return percentile_

In [ ]:
data_imputed['Age'] = 2016 - data_imputed['BirthYear']

In [ ]:
data_imputed.groupby('Labels_KMeans')['Age'].agg(['count', 'mean', 
                                          'median', 'min', 
                                          'max', percentile(25),
                                          percentile(75),
                                          'std'])

In [ ]:
fig = px.box(data_imputed, 
             y='Age', color='Labels_KMeans')
fig.update_layout(legend_title='Cluster',
                  width=700, height=500,
                  xaxis_title='Cluster',
                  yaxis_title='Age')
fig.show()

<div class="alert alert-block alert-info">
<a class="anchor" id="semi_learning"> 
<h1>6. EXTRA - Semi-Supervised Learning</h1>
</a>    
</div>


The labels created with KMeans can be used to predict the group of clients in unseen data from the KMeans model. for that a Supervised Learning algorithm can be used.

First the data is splitted in train and test set.

The data used is the data with original features without 'GeoLivArea', imputed and scaled with z-score.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[data.drop('GeoLivArea', axis=1).columns].astype('float64'), 
                                                    labels_kmeans, test_size=0.2, 
                                                    random_state=42)

The features used are the numerical variables from the original data.

The model will be developed with Xtreme Gradient Boosting using Decision Trees.

In [ ]:
xgbc = XGBClassifier()
xgbc.fit(X_train, y_train)

In [ ]:
y_pred = xgbc.predict(X_test)
features_importances = xgbc.feature_importances_
columns = data.drop('GeoLivArea', axis=1).columns
features_importances = zip(columns, features_importances)
pd.DataFrame(features_importances).sort_values(1, ascending=False)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))

cm_heatmap = px.imshow(cm, text_auto=True,
                       title='Confusion Matrix',
                       color_continuous_scale='rdbu')

cm_heatmap.update_layout(
    yaxis_title="Actual",
    xaxis_title="Predicted",
    autosize=False,
    width=500
)
cm_heatmap.show()

The diagonal values corresponds to correct prediction for each label. 

The labels obtained can be correctly predicted by the XGBoost Classifier.

In [ ]:
print(' -------------------------------------\n'
    f'| F1 Score of the prediciton is: '
    f'{round(f1_score(y_test, y_pred, average="macro", labels=[0, 1, 2]), 2)} |\n'
     ' -------------------------------------')